In [ ]:
#!/usr/bin/env python3
"""
Enhanced Groq Chatbot with Modern UI using Gradio
Supports PDF, Word, Text, CSV, and JSON files
Author: AI Assistant
Version: 4.2
"""

import gradio as gr
import requests
import json
import os
import sys
import tempfile
import shutil
from pathlib import Path
from typing import Optional, List

# New RAG dependencies
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Try to import optional file parsers
try:
    import PyPDF2
    PDF_SUPPORT = True
except ImportError:
    PDF_SUPPORT = False

try:
    import docx
    DOCX_SUPPORT = True
except ImportError:
    DOCX_SUPPORT = False

import csv
from io import StringIO

# Available Groq models
GROQ_MODELS = [
    "llama-3.1-8b-instant",
    "llama-3.1-70b-versatile",
    "llama-3.3-70b-versatile",
    "mixtral-8x7b-32768",
    "gemma2-9b-it",
    "llama3-groq-70b-8192-tool-use-preview",
    "llama3-groq-8b-8192-tool-use-preview"
]

uploaded_file_content = ""
uploaded_file_name = ""
file_type = ""
retriever = None

# Custom CSS for modern UI
custom_css = """
/* Modern Gradio Styling */
.gradio-container {
    max-width: 1200px !important;
    margin: 0 auto !important;
    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif !important;
}

/* Header styling */
.gradio-container .app-header {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 24px;
    border-radius: 16px 16px 0 0;
    text-align: center;
    margin-bottom: 0;
}

.gradio-container .app-header h1 {
    margin: 0;
    font-size: 32px;
    font-weight: 700;
}

.gradio-container .app-header p {
    margin: 8px 0 0 0;
    opacity: 0.9;
    font-size: 16px;
}

/* Main content area */
.gradio-container .main-content {
    background: white;
    border-radius: 0 0 16px 16px;
    box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1);
    padding: 24px;
}

/* File upload section */
.gradio-container .file-upload {
    background: #f8fafc;
    border: 2px dashed #cbd5e1;
    border-radius: 12px;
    padding: 24px;
    text-align: center;
    transition: all 0.3s ease;
    margin-bottom: 24px;
}

.gradio-container .file-upload:hover {
    border-color: #667eea;
    background: #f1f5f9;
}

/* Status and preview styling */
.gradio-container .status-box {
    background: #f0f9ff;
    border-left: 4px solid #0ea5e9;
    padding: 16px;
    border-radius: 8px;
    margin: 16px 0;
}

.gradio-container .preview-box {
    background: #f8fafc;
    border: 1px solid #e2e8f0;
    border-radius: 8px;
    padding: 16px;
    max-height: 200px;
    overflow-y: auto;
    font-family: monospace;
    font-size: 12px;
    line-height: 1.4;
}

/* Settings section */
.gradio-container .settings-section {
    background: #f8fafc;
    border-radius: 12px;
    padding: 20px;
    margin-bottom: 24px;
}

.gradio-container .settings-section h3 {
    margin: 0 0 16px 0;
    color: #1e293b;
    font-size: 18px;
    font-weight: 600;
}

/* Input styling */
.gradio-container input[type="text"],
.gradio-container input[type="password"],
.gradio-container select,
.gradio-container textarea {
    border: 1px solid #d1d5db !important;
    border-radius: 8px !important;
    padding: 12px !important;
    font-size: 14px !important;
    transition: border-color 0.2s !important;
}

.gradio-container input[type="text"]:focus,
.gradio-container input[type="password"]:focus,
.gradio-container select:focus,
.gradio-container textarea:focus {
    border-color: #667eea !important;
    box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1) !important;
    outline: none !important;
}

/* Button styling */
.gradio-container .gr-button {
    background: #667eea !important;
    color: white !important;
    border: none !important;
    border-radius: 8px !important;
    padding: 12px 24px !important;
    font-weight: 500 !important;
    transition: all 0.2s !important;
    cursor: pointer !important;
}

.gradio-container .gr-button:hover {
    background: #5a67d8 !important;
    transform: translateY(-1px) !important;
    box-shadow: 0 4px 12px rgba(102, 126, 234, 0.3) !important;
}

/* Chat response area */
.gradio-container .response-area {
    background: white;
    border: 1px solid #e2e8f0;
    border-radius: 12px;
    padding: 20px;
    min-height: 200px;
    font-size: 14px;
    line-height: 1.6;
}

/* Checkbox and slider styling */
.gradio-container .gr-checkbox {
    accent-color: #667eea !important;
}

.gradio-container .gr-slider input[type="range"] {
    accent-color: #667eea !important;
}

/* Responsive design */
@media (max-width: 768px) {
    .gradio-container {
        margin: 10px !important;
    }
    
    .gradio-container .app-header {
        padding: 16px !important;
    }
    
    .gradio-container .app-header h1 {
        font-size: 24px !important;
    }
    
    .gradio-container .main-content {
        padding: 16px !important;
    }
}
"""

class FileProcessor:
    @staticmethod
    def read_text_file(path):
        for enc in ['utf-8', 'utf-16', 'latin1', 'cp1252']:
            try:
                with open(path, 'r', encoding=enc) as f:
                    return f.read()
            except UnicodeDecodeError:
                continue
        raise Exception("Failed decoding file")

    @staticmethod
    def read_pdf_file(path):
        if not PDF_SUPPORT:
            return "PDF support unavailable"
        with open(path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            return "\n".join([p.extract_text() or "" for p in reader.pages])

    @staticmethod
    def read_docx_file(path):
        if not DOCX_SUPPORT:
            return "DOCX support unavailable"
        doc = docx.Document(path)
        parts = [p.text for p in doc.paragraphs if p.text.strip()]
        return "\n".join(parts)

    @staticmethod
    def read_csv_file(path):
        with open(path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            return "\n".join([" | ".join(row) for row in reader])

    @staticmethod
    def read_json_file(path):
        with open(path, 'r', encoding='utf-8') as f:
            return json.dumps(json.load(f), indent=2, ensure_ascii=False)

def read_file_content(path):
    ext = os.path.splitext(path)[1].lower()
    processor = FileProcessor()
    if ext == ".txt":
        return processor.read_text_file(path), "Text"
    elif ext == ".pdf":
        return processor.read_pdf_file(path), "PDF"
    elif ext in [".docx", ".doc"]:
        return processor.read_docx_file(path), "Word"
    elif ext == ".csv":
        return processor.read_csv_file(path), "CSV"
    elif ext == ".json":
        return processor.read_json_file(path), "JSON"
    return processor.read_text_file(path), "Unknown"

def prepare_rag_context(raw_text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = splitter.create_documents([raw_text])
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = Chroma.from_documents(docs, embedding=embeddings)
    return vectorstore

def call_groq_api(prompt, api_key, model, temperature, rag_db=None):
    if rag_db:
        docs = rag_db.similarity_search(prompt, k=4)
        context = "\n\n".join([doc.page_content for doc in docs])
        prompt = f"""You are an AI assistant helping analyze documents.\n\nCONTEXT:\n{context}\n\nQUESTION:\n{prompt}"""

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    data = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "model": model,
        "temperature": temperature,
        "max_tokens": 2048,
        "stream": False
    }
    try:
        response = requests.post(url, headers=headers, json=data, timeout=30)
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"API Error: {str(e)}"

def handle_file_upload(file):
    global uploaded_file_content, uploaded_file_name, file_type, retriever
    if not file:
        uploaded_file_content = ""
        uploaded_file_name = ""
        file_type = ""
        retriever = None
        return "No file uploaded", "", "📄 No document loaded"
    
    uploaded_file_name = os.path.basename(file.name)
    content, detected = read_file_content(file.name)
    uploaded_file_content = content
    file_type = detected
    retriever = prepare_rag_context(content)
    preview = content[:1000] + ("\n..." if len(content) > 1000 else "")
    
    status_message = f"✅ Successfully loaded: {uploaded_file_name}"
    file_info = f"📄 Document loaded: {uploaded_file_name} ({file_type})"
    
    return status_message, preview, file_info

def groq_interface(prompt, api_key, model, temperature, use_rag):
    if not prompt.strip():
        return "❌ Please enter a prompt."
    
    if not api_key.strip():
        return "❌ Please enter your Groq API key."
    
    if use_rag and not retriever:
        return "❌ Please upload a document to use RAG retrieval."
    
    return call_groq_api(prompt, api_key, model, temperature, retriever if use_rag else None)

if __name__ == "__main__":
    with gr.Blocks(css=custom_css, title="Groq RAG Chatbot") as app:
        # Header
        gr.HTML("""
        <div class="app-header">
            <h1>🚀 Groq RAG Chatbot</h1>
            <p>Intelligent Document Analysis with Advanced AI</p>
        </div>
        """)
        
        with gr.Row():
            # Left column - File upload and settings
            with gr.Column(scale=1):
                gr.HTML("<div class='settings-section'><h3>📄 Document Upload</h3></div>")
                
                file_upload = gr.File(
                    label="Upload Document",
                    file_types=[".txt", ".pdf", ".docx", ".csv", ".json"],
                    type="filepath",
                    elem_classes=["file-upload"]
                )
                
                file_status = gr.Textbox(
                    label="Status",
                    interactive=False,
                    elem_classes=["status-box"]
                )
                
                preview = gr.Textbox(
                    label="Document Preview",
                    lines=8,
                    interactive=False,
                    elem_classes=["preview-box"]
                )
                
                gr.HTML("<div class='settings-section'><h3>⚙️ Settings</h3></div>")
                
                api_key = gr.Textbox(
                    label="Groq API Key",
                    type="password",
                    placeholder="Enter your Groq API key"
                )
                
                model = gr.Dropdown(
                    GROQ_MODELS,
                    value="llama-3.1-8b-instant",
                    label="Model"
                )
                
                temp = gr.Slider(
                    minimum=0,
                    maximum=1,
                    step=0.1,
                    value=0.7,
                    label="Temperature"
                )
                
                use_rag = gr.Checkbox(
                    label="Use RAG Retrieval",
                    value=True
                )
            
            # Right column - Chat interface
            with gr.Column(scale=2):
                gr.HTML("<h3>💬 Chat Interface</h3>")
                
                document_info = gr.Textbox(
                    label="Document Status",
                    value="📄 No document loaded",
                    interactive=False
                )
                
                prompt = gr.Textbox(
                    label="Your Question",
                    lines=3,
                    placeholder="Ask a question about your document..."
                )
                
                response = gr.Textbox(
                    label="AI Response",
                    lines=12,
                    interactive=False,
                    elem_classes=["response-area"]
                )
                
                send = gr.Button("Send Message", variant="primary")
        
        # Event handlers
        file_upload.change(
            fn=handle_file_upload,
            inputs=[file_upload],
            outputs=[file_status, preview, document_info]
        )
        
        send.click(
            fn=groq_interface,
            inputs=[prompt, api_key, model, temp, use_rag],
            outputs=[response]
        )
        
        # Enter key functionality
        prompt.submit(
            fn=groq_interface,
            inputs=[prompt, api_key, model, temp, use_rag],
            outputs=[response]
        )

    app.launch(
        server_name="0.0.0.0",
        server_port=7860,
        inbrowser=True,
        share=False
    )

c:\Users\seifg_nk6r0ks\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\seifg_nk6r0ks\AppData\Local\Temp\ipykernel_13172\4196711404.py:238: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(return_messages=True)
2025-07-09 14:16:59,399 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


TypeError: EventListener._setup.<locals>.event_trigger() got an unexpected keyword argument 'every'